# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [2]:
import pandas as pd
bigrams = pd.read_csv('bigrams.txt', sep='\t', encoding='ansi', header=None)

In [3]:
bigrams.rename(columns={0:'freq',1:'prev_word',2:'word'}, inplace=True)

In [4]:
fivegrams = pd.read_csv('fivegrams.txt', sep='\t', header=None)
fivegrams.rename(columns={0:'freq',1:'prev_word4',2:'prev_word3',3:'prev_word2',4:'prev_word',5:'word'}, inplace=True)

In [48]:
unigrams = Counter(words(open('big.txt').read()))

In [61]:
import re
import nltk
import string
import random

class ContextSpellCorrector:
    def __init__(self, unigrams, bigrams, fivegrams, alpha=1, beta=0.2, gamma=0.1, delta=0.5, sigma=0.5):
        self.bigrams = bigrams
        self.fivegrams = fivegrams
        self.unigrams = unigrams
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.sigma = sigma

    def P_unigram(self, word):
        N = sum(self.unigrams.values())
        return unigrams[word] / N
    
    def P_bigram_inv(self, word, next_word_bigram):
        next_word_bigram = next_word_bigram[-1]
        bigram_word_count = self.bigrams[(self.bigrams.word==next_word_bigram) & (self.bigrams.prev_word==word)].freq.sum()+1
        bigram_next_word_count = self.bigrams[self.bigrams.word==next_word_bigram].freq.sum()+1
        return bigram_word_count / bigram_next_word_count

    def P_bigram(self, word, prev_word_bigram):
        prev_word_bigram = prev_word_bigram[-1]
        bigram_word_count = self.bigrams[(self.bigrams.prev_word==prev_word_bigram) &(self.bigrams.word==word)].freq.sum()+1
        bigram_prev_word_count = self.bigrams[self.bigrams.prev_word==prev_word_bigram].freq.sum()+1
        return bigram_word_count / bigram_prev_word_count

    def P_fivegram(self, word, prev_words):
        # prev_words - 4 words
        prev_word, prev_word2, prev_word3, prev_word4 = prev_words[-1],prev_words[-2],prev_words[-3],prev_words[-4]
        
        fivegram_word_count = self.fivegrams[(self.fivegrams.prev_word==prev_word)&(self.fivegrams.prev_word2==prev_word2)
                                             & (self.fivegrams.prev_word3==prev_word3)
                                             & (self.fivegrams.prev_word4==prev_word4)
                                             & (self.fivegrams.word==word)].freq.sum()+1
        fivegram_prev_word_count = self.fivegrams[(self.fivegrams.prev_word==prev_word)&(self.fivegrams.prev_word2==prev_word2)
                                             & (self.fivegrams.prev_word3==prev_word3)
                                             & (self.fivegrams.prev_word4==prev_word4)].freq.sum()+1
        return fivegram_word_count / fivegram_prev_word_count

    def correct(self, word, prev_words=None, future_word=None, threshold=0.5, debug=False):
        candidates = self.generate_candidates(word)
        if len(prev_words)==5:
            for i in range(len(candidates)):
                candidates[i] = [candidates[i], self.alpha*self.P_unigram(candidates[i])+self.gamma*self.P_fivegram(candidates[i],prev_words)+self.beta*self.P_bigram(candidates[i], prev_words)]
        elif 1<=len(prev_words)<5:
            for i in range(len(candidates)):
                candidates[i] = [candidates[i], self.alpha*self.P_unigram(candidates[i])+self.beta*self.P_bigram(candidates[i],prev_words)]
        else:
            for i in range(len(candidates)):
                candidates[i] = [candidates[i],1/len(candidates)]
        
        if future_word:
            for i in range(len(candidates)):
                prob_inv = self.P_bigram_inv(candidates[i][0], future_word)
                if prob_inv != 1:
                    candidates[i][1] = self.delta*candidates[i][1] + self.sigma*prob_inv
#                 candidates_inv.append((candidates[i][0], ))
#             candidates_inv.sort(reverse=True, key=lambda x:x[-1])
#             if len(candidates_inv)==1:
#                 best_candidate_inv = candidates_inv[0]
#             else:
#                 if candidates_inv[0][-1]==candidates_inv[1][-1]:
                    # we have candidates with similar probs
#                     similar_candidates = [cand for cand in candidates_inv if cand[-1]==candidates_inv[0][-1]]
#                     best_candidate_inv = random.choice(similar_candidates)
#                 else:
                    # we have only one best candidate
#                     best_candidate_inv =  candidates_inv[0]
#             if (best_candidate_inv[1]>threshold) and (best_candidate_inv[1]!=1):
#                 return best_candidate_inv[0]
        candidates.sort(reverse=True, key=lambda x: x[-1])
        if debug:
            print(candidates)
        if len(candidates)==1:
            return candidates[0][0]
        else:
            if candidates[0][-1]==candidates[1][-1]:
                # we have candidates with similar probs
                similar_candidates = [cand[0] for cand in candidates if cand[-1]==candidates[0][-1]]
                return random.choice(similar_candidates)
            else:
                # we have only one best candidate
                return candidates[0][0]

    def generate_candidates(self, word):
        # Generate candidate corrections using edit distance 
        spell_checker = nltk.corpus.words.words()
        candidates = set()
        if word in spell_checker:
            return [word]
        # insert a char
        for i in range(len(word)+1):
            for c in string.ascii_lowercase:
                candidate = word[:i] + c + word[i:]
                if candidate in spell_checker:
                    candidates.add(candidate)
                
        # delete a char
        for i in range(len(word)):
            candidate = word[:i] + word[i+1:]
            if candidate in spell_checker:
                candidates.add(candidate)
        # substitute a char
        for i in range(len(word)):
            for c in string.ascii_lowercase:
                candidate = word[:i] + c + word[i+1:]
                if candidate in spell_checker:
                    candidates.add(candidate)
        if len(candidates)==0:
            return [word]
        return list(candidates)

corrector = ContextSpellCorrector(unigrams, bigrams, fivegrams)

def correct_sentence(sentence, debug=False):
    words = re.findall(r'\b\w+\b', sentence)
    for i in range(len(words)):
        word = words[i]
        prev_words = words[:i]
        future_words = words[i+1:]
        corrected_word = corrector.correct(word, prev_words, future_words, debug)
#         print(corrected_word)
        words[i] = corrected_word
    return ' '.join(words)


## Justify your decisions

Write down justificaitons for your implementation choices.

<b>The reasons for my implementation:</b>
1. The approach uses n-gram models to estimate the probability of a word given its context. It allows the spell checker to consider not only the current word but also its surrounding words.
2. Candidates are generated using edit distances applied to the misspelled word.
3. The spell corrector takes into account both previous and next words when choosing the best correction. It was made to fix such issues as "dking sport" vs "dking species".
4. In cases when multiple candidates have same probabilities, the algorithm chooses the random one, so that it is not stuck with the same correct each run.

<b>Problems I have encountered:</b>
1. I haven't managed to find a good dataset of ngrams for bigrams_inverse approach. Because of this, the algorithm may give wrong corrections. For example, as the bigram dataset doesn't contain a bigram 'doing sport', it won't be able always predict this correction, but instead will randomize the correction for 'dking'.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [62]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [63]:
with open('sentences.txt','r') as f:
    sentences = f.read()

In [64]:
sentences = sentences.split('\n')

In [65]:
sentences = [sentence.replace('вЂ™',"'").replace('.','') for sentence in sentences]

In [66]:
from math import ceil
def misspell_word(word):
    candidates = set()
    for i in range(len(word) + 1):
        for c in string.ascii_lowercase:
            candidate = word[:i] + c + word[i:]
            candidates.add(candidate)
    for i in range(len(word)):
        candidate = word[:i] + word[i + 1:]
        candidates.add(candidate)
    for i in range(len(word)):
        for c in string.ascii_lowercase:
            candidate = word[:i] + c + word[i + 1:]
            candidates.add(candidate)
    return random.choice(list(candidates))
def generate_test_set(dataset, noise):
    test_set = []
    for i in range(len(dataset)):
        # Noise is how many words in a sentence require correction
        original_sent = dataset[i]
        sent = dataset[i].split()
        n = len(sent)
        n_corrections = ceil(n*noise)
        indices_to_change = random.sample(range(len(sent)),k=n_corrections)
        for j in indices_to_change:
            sent[j] = misspell_word(sent[j])
        test_set.append((' '.join(sent),original_sent))
    return test_set

In [67]:
from tqdm import tqdm

In [68]:
def evaluate(test_set):
    my_score = 0
    norvig_score = 0
    for sample in tqdm(test_set):
        sent_to_fix, original_sent = sample
        # My algorithm
        my_sent = correct_sentence(sent_to_fix)
        # Norvig
        norvig_sent = [correction(w) for w in sent_to_fix.split()]
        my_score += sum(1 for a, b in zip(my_sent.split(), sent_to_fix.split()) if a == b)/len(sent_to_fix.split())
        norvig_score += sum(1 for a, b in zip(norvig_sent, sent_to_fix.split()) if a == b)/len(sent_to_fix.split())

    return my_score/len(test_set), norvig_score/len(test_set)

In [69]:
noises = [0.1,0.25,0.5]
for noise in noises:
    test_set = generate_test_set(sentences[:100], noise)
    my_acc,norvig_acc = evaluate(test_set)
    print(f"My accuracy:{my_acc}\t Norvig's accuracy:{norvig_acc}")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [18:45<00:00, 11.25s/it]


My accuracy:0.648727771411595	 Norvig's accuracy:0.7106522329150036


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [25:24<00:00, 15.25s/it]


My accuracy:0.5558984135489553	 Norvig's accuracy:0.6093306944173819


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [32:46<00:00, 19.67s/it]

My accuracy:0.44964906189441806	 Norvig's accuracy:0.45677017134911874


<b>1. Accuracy:</b>
* In the experiments, Norvig's corrector generally outperformed my approach.
* The difference between algorithms is that my checker utilizes n-grams to predict the correction and also considers the future context, while Norvig's corrector only uses word frequencies

<b>2. Flexibility:</b>
* An advantage of my approach is its tunable hyperparamters. We can adjust the following parameters: 'alpha', 'beta', 'gamma', 'delta' and 'sigma'. This can be used to improve the accuracy.